# Disconnect subscribers in OCS and DMI by Soap API request

In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

## Step 1. Import libraries

In [10]:
#python3
#Author: pavel@roamability.com

import numpy as np
import pandas as pd
from pandas import DataFrame,Series
from zeep import Client
from zeep.wsse.username import UsernameToken
import time
from datetime import datetime
import logging
from importlib import reload
from pathlib import Path
import pandas as pd
import pyodbc
from importlib import reload
from sys import exit
reload(logging)

reload(logging)
logger = logging.getLogger('disconnect_subs')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

noc_procedures_log_file = './LOG/noc_procedures.log'

fh = logging.FileHandler(noc_procedures_log_file)
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
logger.addHandler(fh)

version = '0.1.4'

logger.info('Step 1. Import libraries. Start.')

#Check csv file separator
def check_csv_file_separator(file_name):
    with open(file_name) as inf:
        s1=inf.readline()
    if s1.find(';')>0:
        csv_sep = ';'
    elif s1.find(',')>0:
        csv_sep = ','
    else:
        logger.error('Wrong {} file format'.format(file_name))
    return csv_sep

logger.info('Libraries imported successfully.')
logger.info('Programm version: {}'.format(version))
logger.debug('Dirrectory: {}'.format(Path.cwd()))
logger.info('Step 1. End.')

2018-09-19 17:57:35,124 - disconnect_subs - INFO - Step 1. Import libraries. Start.
2018-09-19 17:57:35,125 - disconnect_subs - INFO - Libraries imported successfully.
2018-09-19 17:57:35,127 - disconnect_subs - INFO - Programm version: 0.1.4
2018-09-19 17:57:35,128 - disconnect_subs - DEBUG - Dirrectory: C:\Users\balob\Documents\GITLAB\NOC_Procedures
2018-09-19 17:57:35,130 - disconnect_subs - INFO - Step 1. End.


## Step 2. Precautions before the beginning
##### 1. In the cell below set the connect variable as 'prod' or 'lab' depending of where to connect.<br>
Apply apropriate reseller to username prod_api@roamability.com in OCS Portal.
##### 2. Create a csv file with subscriber_id and imsi without headings. For example:
718755;724540540346789<br>
730932;724540540425234<br>

An example of SQL to generate the file:<br>
USE [10028];<br>
GO<br>
SELECT s.subscriber_id,si.imsi<br>
FROM SUBSCRIBERS s<br>
LEFT JOIN SUBSCRIBER_IMSIS si<br>
ON s.subscriber_id=si.subscriber_id<br>
LEFT JOIN SUBSCRIBER_STATUS ss<br>
ON s.subscriber_id=ss.subscriber_id<br>
WHERE s.account_id=352363 --AccountID<br>
AND ss.end_date IS NULL<br>
AND ss.STATUS = 'Active'<br>

##### 3. Name the file as 'IMSI_list_to_disconnect.csv' and put it in ./DATA folder inthe same dirrectory with this Jupyter notebook.

## Step 3. Pre-Check
#### Choose connection either prod or lab

In [11]:
logger.info('Step 3. Pre-Check. Start.')

connect=input("Choose connection (prod or lab):")
if connect in ['lab','prod']:
    logger.info('Your current connection is: {}'.format(connect))
else:
    logger.error('You have put the wrong connection: {}'.format(connect))
    exit('You have put the wrong connection: {}'.format(connect))
    
imsi_file = '.\DATA\IMSI_list_to_disconnect.csv'

if connect == 'lab':
    user = 'lab_api@roamability.com'
    password = 'E10ADC3949BA59ABBE56E057F20F883E' #Lab
    api_link = 'http://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
elif connect == 'prod':
    user = 'prod_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD'
    api_link = 'http://172.20.35.12:8585/ocsapi/roamability/api/roamability.wsdl'

user_name_token = UsernameToken(user, password)
user_name_token.use_digest = True
logger.debug('Creating connection to OCS by Soap API')
client = Client(api_link, wsse=user_name_token)

logger.debug('Getting Reselle and Account info from OCS by Soap API')
reseller = client.service.getResellerInfo()
accounts = client.service.getAccounts()

if reseller.result.code == '1':
    print('You are connected to Reseller: {}.\
    \nAvailable credit for the Reseller: {:.0f} USD.\n'.\
    format(reseller.reseller.resellerName,reseller.reseller.availableCredit))
else:
    logger.error('Error in getting Reseller info: {}'.format(reseller.result.description))

if accounts.result.code == '1':
    accounts_dict = {account.accountId:account.accountName for account in accounts.accounts.accounts}
    print('The Reseller contains the following accounts:')
    for i,[account_id,account_name] in enumerate(accounts_dict.items()):
        print(' {:<5}{} (accountID: {})'.format(i,account_name,account_id))
else:
    logger.error('Error in getting Account info: {}'.format(accounts.result.description))

logger.debug('Getting Reselle and Account info from OCS by Soap API done')
logger.debug('Start reading file' + imsi_file)

csv_sep = check_csv_file_separator(imsi_file)

try:
    df_imsi = pd.read_csv(imsi_file,sep=';',\
                      names=('subscriber_id','imsi','status','accountId','accountName','new_status','rep_time'))
except:
    logger.error('The file {} was not found or contains errors.'.format(imsi_file))
logger.debug('End reading file' + imsi_file)

if df_imsi[['subscriber_id','imsi']].isnull().values.any():
    logger.error('The file {} has wrong format'.format(imsi_file))
    exit('The file {} has wrong format'.format(imsi_file))
logger.debug('Checking subscribers in OCS by Soap API')

for i in df_imsi['subscriber_id']:
    subscriber = client.service.getSubscriberById(i)
    if subscriber.result.code == '1':
        df_imsi.loc[df_imsi['subscriber_id']==i,['status','accountId','accountName']]=\
        [subscriber.subscriber.status,subscriber.subscriber.accountId,accounts_dict[subscriber.subscriber.accountId]]
    else:
        df_imsi.loc[df_imsi['subscriber_id']==i,['status']]=subscriber.result.description
df_imsi['accountName'].fillna('Unk',inplace=True)
logger.debug('Checking subscribers in OCS by Soap API done')

df1=df_imsi['subscriber_id'].groupby([df_imsi['status'],df_imsi['accountName']]).count()
print('\nGiven list of IMSIs contains:')
total_sibscribers = 0
for i_st,i_ac,v in zip(df1.index.get_level_values('status'),df1.index.get_level_values('accountName'),df1.values):
    print('In account \'{}\' {} subscriber(s) in status \'{}\''.format(i_ac,v,i_st))
    total_sibscribers += v
print('Total subscribers: {}'.format(total_sibscribers))
try:
    df_imsi['accountId'] = df_imsi['accountId'].astype('int64')
except:
    logger.error('The DataFrame has objects in accountId.')
logger.info('Step 3. End.')

2018-09-19 17:57:41,226 - disconnect_subs - INFO - Step 3. Pre-Check. Start.


Choose connection (prod or lab):lab


2018-09-19 17:57:44,151 - disconnect_subs - INFO - Your current connection is: lab
2018-09-19 17:57:44,152 - disconnect_subs - DEBUG - Creating connection to OCS by Soap API
2018-09-19 17:57:44,613 - disconnect_subs - DEBUG - Getting Reselle and Account info from OCS by Soap API
2018-09-19 17:57:45,012 - disconnect_subs - DEBUG - Getting Reselle and Account info from OCS by Soap API done
2018-09-19 17:57:45,017 - disconnect_subs - DEBUG - Start reading file.\DATA\IMSI_list_to_disconnect.csv
2018-09-19 17:57:45,035 - disconnect_subs - DEBUG - End reading file.\DATA\IMSI_list_to_disconnect.csv
2018-09-19 17:57:45,040 - disconnect_subs - DEBUG - Checking subscribers in OCS by Soap API


You are connected to Reseller: SecureGroup RT.    
Available credit for the Reseller: -0 USD.

The Reseller contains the following accounts:
 0    Secure Groip Sim-cards (accountID: 352363)


2018-09-19 17:57:46,812 - disconnect_subs - DEBUG - Checking subscribers in OCS by Soap API done
2018-09-19 17:57:46,828 - disconnect_subs - INFO - Step 3. End.



Given list of IMSIs contains:
In account 'Secure Groip Sim-cards' 12 subscriber(s) in status 'Active'
Total subscribers: 12


## Step 4. Installation
Run this step **ONLY** if all previous steps completed with success.<br>

In [12]:
start_time = datetime.now()
logger.info('Step 4. Installation. Start.')

modify_agreement=input('You are going to disconnest the subscribers in {}. Type Y to continue:'.format(connect))
if modify_agreement != 'Y':
    logger.info('Stop executing Step 4.')
    exit('Stop executing Step 4.')

for j,subscriber_id in enumerate(df_imsi[df_imsi['status']=='Active']['subscriber_id']):
    
    try:
        disc_subscriber = client.service.disconnectSubscriber(subscriber_id)
        #disc_subscriber = client.service.getSubscriberById(subscriber_id)
    except:
        logger.error('The procedure ended with an unknown error on the row: {}'.format(j))
        log_file_name = '.\LOG\Log_Disconnect_sub_' + datetime.now().strftime("%Y%m%d%H%M%S") + '_error.csv'
        df_msisdn.to_csv(log_file_name,index=False)
        logger.info('Log file {} saved with success.'.format(log_file_name))
        exit('Stop executing Step 4 with error.')
    if disc_subscriber.result.code == '1':
        df_imsi.loc[df_imsi['subscriber_id']==subscriber_id,['new_status','rep_time']]=\
        [disc_subscriber.subscriber.status,datetime.now()]
    else:
        df_imsi.loc[df_imsi['subscriber_id']==subscriber_id,['new_status','rep_time']]=\
        [disc_subscriber.result.description,datetime.now()]
    if ((j+1) % 10) == 0:
        print('{:<5} Subscribers processed elapsed time {}'.format(j+1,datetime.now() - start_time))
    if ((j+1) % 100) == 0:
        print('Sleep for 10 sec.')
        time.sleep(10)
logger.info('All done. {} subscribers processed.\nTotal elapsed time {}'.format(j+1,datetime.now() - start_time))
log_file_name = '.\LOG\Log_Disconnect_sub_' + datetime.now().strftime("%Y%m%d%H%M%S") + '.csv'
logger.info('Log file {} saved with success.'.format(log_file_name))
df_imsi.to_csv(log_file_name,index=False)
print('The first 5 rows of the log list:')
display(df_imsi.head())
logger.info('Step 4. End.')
%reset -f

2018-09-19 17:57:58,105 - disconnect_subs - INFO - Step 4. Installation. Start.


You are going to disconnest the subscribers in lab. Type Y to continue:Y
10    Subscribers processed elapsed time 0:00:03.861717


2018-09-19 17:58:02,237 - disconnect_subs - INFO - All done. 12 subscribers processed.
Total elapsed time 0:00:04.131993
2018-09-19 17:58:02,238 - disconnect_subs - INFO - Log file .\LOG\Log_Disconnect_sub_20180919175802.csv saved with success.


The first 5 rows of the log list:


,subscriber_id,imsi,status,accountId,accountName,new_status,rep_time
0,209418,425019613000620,Active,352363,Secure Groip Sim-cards,Active,2018-09-19 17:58:00.673695
1,209420,425019613000622,Active,352363,Secure Groip Sim-cards,Active,2018-09-19 17:58:00.813358
2,209422,425019613000624,Active,352363,Secure Groip Sim-cards,Active,2018-09-19 17:58:00.995859
3,209424,425019613000626,Active,352363,Secure Groip Sim-cards,Active,2018-09-19 17:58:01.129515
4,209426,425019613000628,Active,352363,Secure Groip Sim-cards,Active,2018-09-19 17:58:01.270130


2018-09-19 17:58:02,261 - disconnect_subs - INFO - Step 4. End.
